In [131]:
import numpy as np
import pandas as pd
from collections import defaultdict
from hvplot import pandas
from typing import Callable, List, Tuple, Dict

In [132]:
# стандартные функции принадлежностей
def trapezoid(a: float, b: float, c: float, d: float) -> Callable[[float], float]:
  def func(x: float) -> float:
    if x < a:   return 0
    elif x < b: return (x - a) / (b - a)
    elif x < c: return 1
    elif x < d: return (d - x) / (d - c)
    else:       return 0
  return func

def left_trapezoid(c: float, d: float) -> Callable[[float], float]:
  def func(x: float) -> float:
    if x < c:   return 1
    elif x < d: return (d - x) / (d - c)
    else:       return 0
  return func

def right_trapezoid(a: float, b: float) -> Callable[[float], float]:
  def func(x: float) -> float:
    if x < a:   return 0
    elif x < b: return (x - a) / (b - a)
    else:       return 1
  return func

In [133]:
# функция поиска центра тяжести функции на заданном интервале
def barycenter(func: Callable[[float], float], start: float, end: float, npoints: int = 100) -> float:
  area = 0
  weighted_area = 0
  for x in np.linspace(start, end, npoints):
    area += func(x)
    weighted_area += x * func(x)
  assert area != 0
  return weighted_area / area

In [134]:
# класс терма лингвистической переменной
class Term:
  def __init__(self, name: str, membership_fn: Callable[[float], float]):
    self.name = name
    self.membership_fn = membership_fn
  
  def __call__(self, value: float) -> float:
    return self.membership_fn(value)


# класс лингвистической переменной
class LinVar:
  def __init__(self, name: str, min_val: float, max_val: float, terms: List[Term]):
    self.name = name
    self.min_val = min_val
    self.max_val = max_val
    self.terms = terms
  
  def fuzzyfy(self, value: float) -> Dict[str, float]:
    return defaultdict(lambda: 0, {term.name : term(value) for term in self.terms if term(value) > 0})
  
  def defuzzyfy(self, terms: Dict[str, float]) -> float:
    # объединенная функция принадлежности
    def comb_memb_fn(x: float) -> float:
      return max(min(terms[term.name], term.membership_fn(x)) for term in self.terms)
    # вычисление центра тяжести объединенной функции принадлежности
    return barycenter(comb_memb_fn, self.min_val, self.max_val)


# класс правила
class Rule:
  def __init__(self, inputs: List[Tuple[str, str]], output: Tuple[str, str]):
    self.inputs = inputs
    self.output = output
  
  def activization(self, var_terms: Dict[str, Dict[str, float]]) -> float:
    # активизация возможна только если известны термы для всех входных переменных
    if not all(var in var_terms for var, _ in self.inputs):
      return 0
    return min(var_terms[var][term] for var, term in self.inputs)

In [135]:
# отображение графиков функций принадлежности термов лингвистической переменной
def show_lin_var(var: LinVar, start_value: float, stop_value: float, steps: int = 300):
  X = np.linspace(start_value, stop_value, steps)
  vec = lambda f: np.vectorize(f, otypes=[float])
  df = pd.DataFrame({term.name: vec(term.membership_fn)(X) for term in var.terms}, index=X)
  return df.hvplot()

In [136]:
lab_uniqueness = LinVar('lab_uniqueness', 0, 100, [
  Term('low', left_trapezoid(22, 45)),
  Term('medium', trapezoid(22, 45, 70, 82)),
  Term('high', right_trapezoid(70, 82)),
])

show_lin_var(lab_uniqueness, 0, 100)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [137]:
theme_knowledge = LinVar('theme_knowledge', 0, 100, [
  Term('low', left_trapezoid(16, 40)),
  Term('medium', trapezoid(16, 40, 48, 85)),
  Term('high', right_trapezoid(48, 85)),
])

show_lin_var(theme_knowledge, 0, 100)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [138]:
# не нормализованный вариант
# lab_mark = LinVar('lab_mark', 0, 100, [
#   Term('low', left_trapezoid(0, 50)),
#   Term('medium', trapezoid(35, 60, 60, 85)),
#   Term('high', right_trapezoid(60, 100)),
# ])

# нормализованный вариант
lab_mark = LinVar('lab_mark', 0, 100, [
  Term('low', left_trapezoid(15, 50)),
  Term('medium', trapezoid(15, 50, 60, 100)),
  Term('high', right_trapezoid(60, 100)),
])

show_lin_var(lab_mark, 0, 100)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [139]:
# определение правил нечеткой логики
rules = [
  Rule([('lab_uniqueness', 'low'),    ('theme_knowledge', 'low')],    ('lab_mark', 'low')),
  Rule([('lab_uniqueness', 'low'),    ('theme_knowledge', 'medium')], ('lab_mark', 'medium')),
  Rule([('lab_uniqueness', 'low'),    ('theme_knowledge', 'high')],   ('lab_mark', 'high')),
  Rule([('lab_uniqueness', 'medium'), ('theme_knowledge', 'low')],    ('lab_mark', 'low')),
  Rule([('lab_uniqueness', 'medium'), ('theme_knowledge', 'medium')], ('lab_mark', 'medium')),
  Rule([('lab_uniqueness', 'medium'), ('theme_knowledge', 'high')],   ('lab_mark', 'high')),
  Rule([('lab_uniqueness', 'high'),   ('theme_knowledge', 'low')],    ('lab_mark', 'medium')),
  Rule([('lab_uniqueness', 'high'),   ('theme_knowledge', 'medium')], ('lab_mark', 'high')),
  Rule([('lab_uniqueness', 'high'),   ('theme_knowledge', 'high')],   ('lab_mark', 'high')),
]

In [145]:
# процедура нечеткого вывода
def fuzzy_solve(*, output_var: LinVar = None, input_var_values: List[Tuple[LinVar, float]] = []) -> float:
  # этап фаззификации
  var_terms = {var.name: var.fuzzyfy(value) for var, value in input_var_values}

  # этап активизации
  active_rules = list(filter(lambda rule:
                             rule.activization(var_terms) > 0 and
                             rule.output[0] == output_var.name, rules))

  # этап композиции
  output_terms = defaultdict(lambda: 0)
  for rule in active_rules:
    output_terms[rule.output[1]] += rule.activization(var_terms)

  # этап дефаззификации
  output_value = output_var.defuzzyfy(output_terms)

  return output_value

fuzzy_solve(input_var_values=[
  (lab_uniqueness, 80),
  (theme_knowledge, 60),
], output_var=lab_mark)

74.50399634979075

In [141]:
# сгенерировать множество всех возможных входов - выходов по реализованному алгоритму
def gen_df(*, lab_uniqueness_steps=5, theme_knowledge_steps=5):
  full_df = {'lab_uniqueness': [], 'theme_knowledge': [], 'lab_mark': []}
  for lab_uniqueness_value in np.linspace(0, 100, lab_uniqueness_steps):
    for theme_knowledge_value in np.linspace(0, 100, theme_knowledge_steps):
      lab_mark_value = fuzzy_solve(input_var_values=[
        (lab_uniqueness, lab_uniqueness_value),
        (theme_knowledge, theme_knowledge_value),
      ], output_var=lab_mark)
      full_df['lab_uniqueness'].append(lab_uniqueness_value)
      full_df['theme_knowledge'].append(theme_knowledge_value)
      full_df['lab_mark'].append(lab_mark_value)
  return pd.DataFrame(full_df)

In [142]:
gen_df(lab_uniqueness_steps=100).hvplot(x='lab_uniqueness', y='lab_mark', by='theme_knowledge')

:NdOverlay   [theme_knowledge]
   :Curve   [lab_uniqueness]   (lab_mark)

In [143]:
gen_df(theme_knowledge_steps=100).hvplot(x='theme_knowledge', y='lab_mark', by='lab_uniqueness')

:NdOverlay   [lab_uniqueness]
   :Curve   [theme_knowledge]   (lab_mark)